In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
import cdsapi
import xarray as xr
from urllib.request import urlopen
warnings.filterwarnings('ignore')

In [ ]:
def get_era_5(ds):
    era_5_df = ds.to_dataframe()
    #MEMORY
    ds = 0
    
    # Can i do this with the meshgrid??
    #era_5_df = era_5_df.dropna()
    
    index_long = era_5_df.index.levels[0]
    index_lat = era_5_df.index.levels[1]
    index_time = era_5_df.index.levels[2]
    u10 = era_5_df["u10"].to_numpy()
    v10 = era_5_df["v10"].to_numpy()
    
    #MEMORY
    era_5_df = 0
    
    start_time = pd.Timestamp(1992, 10, 5)
    index_time_fixed = []
    index_date_fixed = []
    for tid in index_time:
        hours = tid - start_time
        index_time_fixed.append(hours.days * 24 + hours.seconds / 3600)
        index_date_fixed.append(tid.date().strftime('%Y_%m_%d'))
        
    #MEMORY
    index_time = 0

    lon, lat, time = np.meshgrid(index_long, index_lat, index_time_fixed, indexing='ij')

    lon, lat, date = np.meshgrid(index_long, index_lat, index_date_fixed, indexing='ij')
    
    #MEMORY 
    index_long = 0
    index_lat = 0
    index_time_fixed = 0
    index_date_fixed = 0
    
    lon = lon.flatten()
    lat = lat.flatten()
    time = time.flatten()
    date = date.flatten()
    
        
    
    df = pd.DataFrame({'lon': lon, 'lat': lat, "time" : time,'date' : date, 'u10': u10,'v10': v10})
    
    # MEMORY 
    lon = 0
    lat = 0
    time = 0
    date = 0
    v10 = 0
    u10 = 0
    
    df['lon'] = np.where(df['lon'] < 0, df['lon'] + 360, df['lon'])
    
    
    
    df_dict = dict(tuple(df.groupby("date")))
    #MEMORY
    df = 0
    
    
    for key in df_dict.keys():
        df_dict[key].drop('date', axis = 1, inplace = True)
        df_dict[key].to_csv("era_5/" + key +".csv" ,index=False)
        #MEMORY 
        df_dict[key] = pd.DataFrame()

In [ ]:
c = cdsapi.Client()
# dataset to read
dataset = 'reanalysis-era5-single-levels'
years = ['2021', '2022']
group_of_days = [
       ['01', '02', '03','04', '05', '06','07', '08', '09','10'],
       ['11', '12','13', '14', '15''16', '17', '18','19', '20'],
       ['21','22', '23', '24','25', '26', '27','28', '29', '30','31']
       ]
for i in tqdm(range(len(years))):
    if years[i] == '2022':
        months = ['01']
    else:
        months = ['09',
            '10', '11', '12',
        ]
        
    for month in months:
        for group in group_of_days:
            if years[i] == '2017' and month == '08' and group == ['01', '02', '03','04', '05', '06','07', '08', '09','10']:
                print("skip")
            else:
                params = {'product_type': 'reanalysis',
                        'variable': ['10m_u_component_of_wind', '10m_v_component_of_wind',],
                        'year': years[i],
                        'month': month,
                        'day': group,
                        'time': [
                            '00:00', '01:00', '02:00',
                            '03:00', '04:00', '05:00',
                            '06:00', '07:00', '08:00',
                            '09:00', '10:00', '11:00',
                            '12:00', '13:00', '14:00',
                            '15:00', '16:00', '17:00',
                            '18:00', '19:00', '20:00',
                            '21:00', '22:00', '23:00',
                        ],
                        'area': [38, -180, -38,180,],
                        'format': 'netcdf',
                    }
                # retrieves the path to the file
                fl = c.retrieve(dataset, params)
                with urlopen(fl.location) as f:
                    get_era_5(xr.open_dataset(f.read()))